In [1]:
from KeRLas import defaultQModel, Brain, TimedGymEnv
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from cartpole import CartPoleEnv

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(CartPoleEnv(), tlimit=300)

space = env.observation_space
high = np.array(
    [
        2.4,
        1.0,
        12 * 2 * math.pi / 360,
        1.0
    ]
)
env.RandomObservationSpace = gym.spaces.Box(-high, high)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n

rlmodel = DirectDiffModel(defaultQModel(observation_width, nactions), 0.9)
policy = BoltzmannQPolicy(1.0)
brain = Brain(env, rlmodel, policy, 10000, 0.5)

mbsize = 50
#for i in range(10):
#    next(brain.Memory.generate_samples(20))
_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [3]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, active_agents, infos, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record):
        env.render()

from KeRLas import GymPlayer

p = GymPlayer(env, brain, callback=Callback())

temps = [10.0, 1.0, 0.1, 0.01, 0.001, 0.0001]

for i in xrange(20):
    training_tau = temps[i%len(temps)]
    policy.tau = training_tau
    brain.training_model().fit_generator(
        brain.trainig_data_generator(mbsize),
        steps_per_epoch = 100, epochs = 20, verbose = False, workers = 0
    )
    nsum = 0
    N = 5
    policy.tau = 0.0001
    for _ in range(N):
        history = p.runEpisode()
        #print len(history)
        nsum += len(history)
    print training_tau, float(nsum)/N

TypeError: onStep() takes exactly 6 arguments (5 given)

In [ ]:
for _ in xrange(100):
    p.runEpisode()